## Data Process and RFE (SVC, LR, DT)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# DT
cl = DecisionTreeClassifier()

In [ ]:
# SVM
cl = SVC(kernel="linear")

In [ ]:
rfecv = RFECV(estimator=cl, step=1, cv=StratifiedKFold(10), scoring='accuracy')

In [ ]:
# to separate labels and features
labels = df_m['Label']
ftr = df_m.drop(['Label'], axis=1)

In [ ]:
# RFE
rfecv.fit(ftr, labels)

In [ ]:
rank_list = pd.DataFrame(data=rfecv.ranking_)
rank_list.to_csv('./data-after-select/rank/ftr-rank-UCEC.csv') # 注意这是 UCEC 的结果

In [ ]:
import time
import multiprocessing
import os

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cas = ['BLCA', "BRCA", "HNSC", "KIRC", "LUAD", "LUSC", "UCEC"]

In [ ]:
def read_ca_file(ca_name):
    ca_fi = "./orig-data/{}/{}-Ca.txt".format(ca_name, ca_name)
    cas_fi = "./orig-data/{}/{}-CaS.txt".format(ca_name, ca_name)
    df_ca = pd.read_csv(ca_fi, sep='\t', index_col='Tags')
    df_cas = pd.read_csv(cas_fi, sep='\t', index_col='Tags')
    return df_ca, df_cas

In [ ]:
def process(df_a, df_b):
    df_a = df_a.T
    df_b = df_b.T
    n_a, n_b = df_a.shape[0], df_b.shape[0]
    df_merged = pd.concat([df_a, df_b])
    print('Total samples: {}'.format(df_merged.shape))
    df_merged = filter_avg(df_merged.T).T
    df_merged['Label'] = 0
    df_merged['Label'].iloc[: n_a] = 1
    # 保存
    return df_merged

In [ ]:
def filter_avg(df, thres=1.0):
    print("before filtering: {}".format(df.shape))
    row_avg = df.mean(axis=1)
    good_avg = df.loc[row_avg > thres]
    print("after filtering: {}".format(good_avg.shape))
    return good_avg

In [ ]:
def process_rfe(ca_name):
    print('Dataset: {}, module name: {}, parent process: {}, process id: {}'
          .format(ca_name, __name__, os.getppid(), os.getpid()))
    df_ca, df_cas = read_ca_file(ca_name)
    df_m = process(df_ca, df_cas)

    labels = df_m['Label']
    ftr = df_m.drop(['Label'], axis=1)
    # RFE 
#     svc = SVC(kernel="linear")
    # DT
    cl = DecisionTreeClassifier()
    rfecv = RFECV(estimator=cl, step=1, cv=StratifiedKFold(10), scoring='accuracy')
    ts = time.time()
    rfecv.fit(ftr, labels)
    print('RFE for {}: time elapsed: {}s'.format(ca_name, time.time() - ts))

    rank_list = pd.DataFrame(data=rfecv.ranking_)
    rank_list.to_csv('./data-after-select/ftr-rank-{}.csv'.format(ca_name))
    with open('time-for-running-{}-is-{}s'.format(ca_name, time.time() - ts), 'w') as f:
        pass

In [ ]:
for ca_name in cas:
    p = multiprocessing.Process(target=process_rfe, args=(ca_name,))
    p.start()